In [ ]:
# Instalando o scikit-surprise
!pip3 install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import wordnet
from functools import reduce 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate, KFold

import warnings; warnings.simplefilter('ignore')

# VEJA ESSE LINK
# https://www.kaggle.com/code/sohamohajeri/movie-recommendation-systems
# https://www.kaggle.com/code/erikbruin/movie-recommendation-systems-for-tmdb/report


# **RECOMENDADOR BASEADO EM CONTEÚDO**

In [ ]:
# Carregando o dataset movies_metadata
md_filmes = pd.read_csv('movies_metadata.csv')
# Exibindo metadados dos filmes
md_filmes

In [ ]:
md_filmes['genres'] = md_filmes['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance (x, list) else [])

In [ ]:
md_filmes['year'] = pd.to_datetime(md_filmes['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [ ]:
md_filmes.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


In [ ]:
# Carregando dataset links_small
links_small = pd.read_csv('links_small.csv')
# Converte o tipo da coluna, filtra os não nulos e cria uma serie com os valores de tmdbId
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
# Exibindo a serie 
links_small

0          862
1         8844
2        15602
3        31357
4        11862
         ...  
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int64

In [ ]:
# Essas linhas possuem dados maus formatados, por isso foram removidas
md_filmes = md_filmes.drop([19730, 29503, 35587])
# Exibindo metadados dos filmes
md_filmes

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,NaT
45462,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,2011
45463,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,1917


In [ ]:
# Convete o tipo da coluna 
md_filmes['id'] = md_filmes['id'].astype('int')

In [ ]:
# Colete todos os metadados para filmes no conjunto de dados links_small
# Similar movie data - smd
smd = md_filmes[md_filmes['id'].isin(links_small)]
smd.shape

(9099, 25)

### ***RECOMENDADOR BASEADO NA DESCRIÇÃO DO FILME***

In [ ]:
# Processando a decrição e slogans dos filmes
# .fillna -> Preencha os valores NA/NaN usando o método especificado.
# Nesse caso preenche em branco
smd['tagline'] = smd['tagline'].fillna('')
# Criando uma coluna de descrição, concatenando conteúdo de 
# overview e tagline
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [ ]:
# Conte a frequência das palavras da descrição e dê uma pontuação  
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')

# .fit_transform-> Aprenda vocabulário e idf, retorne a matriz de termo de documento.
#                  retorna uma matriz de termo de documento ponderada por Tf-idf
tfidf_matrix = tf.fit_transform(smd['description'])
tfidf_matrix.shape

(9099, 268124)

In [ ]:
# Aqui vamos iniciar o calculo da similaridade de cosseno
# linear_kernel -> Calcule o kernel linear entre X e Y.
# Encontrando semelhança de cosseno usando LinearKernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
# Agora o passo é implementar a função que retorna os 30 filmes 
# mais semelhantes com base na pontuação de similaridade do coseno
# Cria uma coluna index em smd
smd = smd.reset_index()
# titles é uma serie que recebe dados da coluna title de SMD
titles = smd['title']
# Crie uma nova série baseada em títulos de filmes
# Construir um mapa reverso de índices e títulos de filmes
indices = pd.Series(smd.index, index=smd['title'])

In [ ]:
# Função que recebe o título do filme como entrada e produz a maioria dos filmes semelhantes
# Ela Recomenda com base nas pontuações de similaridade
def obter_recomendacoes(title):
  # Obtenha o índice do filme que corresponde ao título
  idx = indices[title]
  # Obtenha as pontuações de semelhança em pares de todos os filmes com esse filme
  # Retorna uma lista de tuplas 
  sim_scores = list(enumerate(cosine_sim[idx]))
  # SÓ A TITULO DE CURIOSIDADE CADA POSIÇÃO EQUIVALE A UM FILME CORRETO?
  # print(sim_scores)
  # print()
  # Classifique os filmes com base nas pontuações de semelhança
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  # print(sim_scores)
  # print()
  # Obtenha as pontuações dos 30 filmes mais parecidos
  sim_scores = sim_scores[1:31]
  # print(sim_scores)
  # print()
  # Obtenha os índices dos filmes
  # Como a lista em sim_scores vem em tupla, estou selecionando para cada item de sim_scores i[0]
  movie_indices = [i[0] for i in sim_scores]
  # print(movie_indices)
  # print()
  # Retorne o top 30 filmes mais similares
  # Retorna o indice e o titulo dos valores da serie que são equivalentes a lista de movie_indices
  return titles.iloc[movie_indices]

In [ ]:
obter_recomendacoes('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [ ]:
obter_recomendacoes('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

### ***RECOMENDADOR BASEADO EM METADADOS DOS FILMES***

In [ ]:
# Carregando dados dos credits  
credits = pd.read_csv('credits.csv')
# Carregando dados dos keywords  
keywords = pd.read_csv('keywords.csv')

In [ ]:
# Converte o tipo da coluna para int
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md_filmes['id'] = md_filmes['id'].astype('int')

In [ ]:
md_filmes.shape

(45463, 25)

In [ ]:
# Fazendo a junção dos Dataframes de filmes com creditos e palavras chave 
md_filmes = md_filmes.merge(credits, on='id')
md_filmes = md_filmes.merge(keywords, on='id')
# md_filmes.shape

In [ ]:
md_filmes

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46623,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,NaT,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...","[{'id': 10703, 'name': 'tragic love'}]"
46624,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,Released,NaN,Century of Birthing,False,9.0,3.0,2011,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...","[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46625,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",[]
46626,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,Released,NaN,Satan Triumphant,False,0.0,0.0,1917,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",[]


In [ ]:
# Colete todos os metadados para filmes no conjunto de dados links_small
# Filtre os filmes cujo id está presente no arquivo links.csv e armazene em smd
# .isin -> Se cada elemento no DataFrame está contido em valores.
# Similar movie data - smd
smd = md_filmes[md_filmes['id'].isin(links_small)]
smd.shape

(9219, 28)

In [ ]:
smd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40952,False,NaN,8000000,[Drama],NaN,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,Released,NaN,The Last Brickmaker in America,False,7.0,1.0,2001,"[{'cast_id': 1, 'character': 'Henry Cobb', 'cr...","[{'credit_id': '544475aac3a36819fb000578', 'de...","[{'id': 6054, 'name': 'friendship'}, {'id': 20..."
41172,False,NaN,1000000,"[Thriller, Romance]",NaN,392572,tt5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,2016,"[{'cast_id': 0, 'character': 'Rustom Pavri', '...","[{'credit_id': '5951baf692514129c4016600', 'de...","[{'id': 10540, 'name': 'bollywood'}]"
41225,False,NaN,15050000,"[Adventure, Drama, History, Romance]",NaN,402672,tt3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,Released,NaN,Mohenjo Daro,False,6.7,26.0,2016,"[{'cast_id': 0, 'character': 'Sarman', 'credit...","[{'credit_id': '57cd5d3592514179d50018e8', 'de...","[{'id': 10540, 'name': 'bollywood'}]"
41391,False,NaN,15000000,"[Action, Adventure, Drama, Horror, Science Fic...",NaN,315011,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0,2016,"[{'cast_id': 4, 'character': 'Rando Yaguchi : ...","[{'credit_id': '560892fa92514177550018b2', 'de...","[{'id': 1299, 'name': 'monster'}, {'id': 7671,..."


In [ ]:
smd['cast'] = smd['cast'].apply(literal_eval)

In [ ]:
smd['crew'] = smd['crew'].apply(literal_eval)

In [ ]:
smd['keywords'] = smd['keywords'].apply(literal_eval)

In [ ]:
# Cria nova coluna com tamanho do cast
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))

In [ ]:
# Cria nova coluna com tamanho do crew
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [ ]:
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",13,106
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",26,16
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",7,4
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",10,10
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",12,7


In [ ]:
def obter_diretor(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']
  return np.nan

In [ ]:
smd['director'] = smd['crew'].apply(obter_diretor)

In [ ]:
# Organiza o nome do cast
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
# Seleciona os 3 principais atores 
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [ ]:
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,False,7.7,5415.0,1995,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",13,106,John Lasseter
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,False,6.9,2413.0,1995,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",26,16,Joe Johnston
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,False,6.5,92.0,1995,"[Walter Matthau, Jack Lemmon, Ann-Margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",7,4,Howard Deutch
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,False,6.1,34.0,1995,"[Whitney Houston, Angela Bassett, Loretta Devine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",10,10,Forest Whitaker
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,False,5.7,173.0,1995,"[Steve Martin, Diane Keaton, Martin Short]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",12,7,Charles Shyer


In [ ]:
smd['keywords'][1]

[{'id': 10090, 'name': 'board game'},
 {'id': 10941, 'name': 'disappearance'},
 {'id': 15101, 'name': "based on children's book"},
 {'id': 33467, 'name': 'new home'},
 {'id': 158086, 'name': 'recluse'},
 {'id': 158091, 'name': 'giant insect'}]

In [ ]:
# Organiza as palavras chaves numa lista
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []) 

In [ ]:
# Remove os espaços e deixa tudo minusculo no cast
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
# Converte o tipo da coluna do director para string, pega o nome do diretor deixa minusculo e sem espaços
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [ ]:
# Repete 3 vezes o nome do diretor na lista para dar mais peso em relação a todo elenco
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

#### **PALAVRAS-CHAVE**  
Será feito um pré-processamento das palavras chaves

In [ ]:
smd

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,False,7.7,5415.0,1995,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",13,106,"[johnlasseter, johnlasseter, johnlasseter]"
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,False,6.9,2413.0,1995,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",26,16,"[joejohnston, joejohnston, joejohnston]"
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,False,6.5,92.0,1995,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o...",7,4,"[howarddeutch, howarddeutch, howarddeutch]"
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,False,6.1,34.0,1995,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",10,10,"[forestwhitaker, forestwhitaker, forestwhitaker]"
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,False,5.7,173.0,1995,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug...",12,7,"[charlesshyer, charlesshyer, charlesshyer]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40952,False,NaN,8000000,[Drama],NaN,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,False,7.0,1.0,2001,"[sidneypoitier, wendycrewson, jayo.sanders]","[{'credit_id': '544475aac3a36819fb000578', 'de...","[friendship, brick making]",7,2,"[greggchampion, greggchampion, greggchampion]"
41172,False,NaN,1000000,"[Thriller, Romance]",NaN,392572,tt5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,False,7.3,25.0,2016,"[akshaykumar, ileanad'cruz, eshagupta]","[{'credit_id': '5951baf692514129c4016600', 'de...",[bollywood],14,16,"[tinusureshdesai, tinusureshdesai, tinusureshd..."
41225,False,NaN,15050000,"[Adventure, Drama, History, Romance]",NaN,402672,tt3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,False,6.7,26.0,2016,"[hrithikroshan, poojahegde, kabirbedi]","[{'credit_id': '57cd5d3592514179d50018e8', 'de...",[bollywood],12,16,"[ashutoshgowariker, ashutoshgowariker, ashutos..."
41391,False,NaN,15000000,"[Action, Adventure, Drama, Horror, Science Fic...",NaN,315011,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,False,6.6,152.0,2016,"[hirokihasegawa, yutakatakenouchi, satomiishih...","[{'credit_id': '560892fa92514177550018b2', 'de...","[monster, godzilla, giant monster, destruction...",49,27,"[hideakianno, hideakianno, hideakianno]"


In [ ]:
# Criar uma serie palavras chave aplicando ao longo do axis=1 coluna
# stack ->retornar um dataframe remodelado, se as colunas tiverem um único nível, a saída é uma Série;
# Reseta index em 1 nível
palavras_chave = smd.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)
# Isso aqui ficou um pouco dificil de entender 

In [ ]:
# Nomeia a serie
palavras_chave.name = 'keyword'

In [ ]:
# Retorna uma série contendo contagens de valores exclusivos.
palavras_chave = palavras_chave.value_counts()

#Exibindo as 5 palavras que ocorrem com mais frequencia  
palavras_chave[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [ ]:
# Selecionando somente as palavras que ocorrem mais de uma vez
palavras_chave = palavras_chave[palavras_chave > 1]

In [ ]:
# Converterndo palavras em seu radical
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [ ]:
def filtrar_palavras_chave(x):
  words = []
  for i in x:
    if i in palavras_chave:
      words.append(i)
  return words

In [ ]:
# Aplicando a função filtrar palavras chaves
smd['keywords'] = smd['keywords'].apply(filtrar_palavras_chave)

In [ ]:
# Aplica a lisa de palavras o stemmer que deixa somente o radical
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

In [ ]:
# Deixa as palavras minusculas e remove os espaços em palavras compostas 
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
# Criando sopa de palavras unindo as colunas 
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']

In [ ]:
# Une as palavras da lista separando por espaços
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [ ]:
# Ele é usado para transformar um determinado texto em um vetor com base na frequência (contagem) 
# de cada palavra que ocorre em todo o texto.
count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
# fit_transform -> Aprenda o dicionário de vocabulário e retorne a matriz documento-termo.
#       Isso é equivalente a fit seguido de transformação, mas implementado de forma mais eficiente.
count_matrix = count.fit_transform(smd['soup'])

In [ ]:
# Calcule a similaridade de cosseno entre as amostras em X e Y.
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Cria uma coluna index contendo os indices do df - reset_index() -> Redefina o índice ou um nível dele.
smd = smd.reset_index()
# Cria uma serie com os titulos 
titles = smd['title']
# Faço o mapa reverso onde o id é o titulo e os valores são o id do filme
# Series -> Array unidimensional com rótulos de eixo (incluindo séries temporais).
indices = pd.Series(smd.index, index=smd['title'])

In [ ]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
The Last Brickmaker in America                        9214
Rustom                                                9215
Mohenjo Daro                                          9216
Shin Godzilla                                         9217
The Beatles: Eight Days a Week - The Touring Years    9218
Length: 9219, dtype: int64

In [ ]:
obter_recomendacoes('Mean Girls').head(10)

3319               Head Over Heels
4763                 Freaky Friday
1329              The House of Yes
6277              Just Like Heaven
7905         Mr. Popper's Penguins
7332    Ghosts of Girlfriends Past
6959     The Spiderwick Chronicles
8883                      The DUFF
6698         It's a Boy Girl Thing
7377       I Love You, Beth Cooper
Name: title, dtype: object

#### **POPULARIDADE E CLASSIFICAÇÃO**  
Devido ao fato do sistema de recomendação recomendar os filmes independente da classificação e popularidade, será feito um processamento para remover filmes ruins e retornar filmes populares e que tiveram uma boa resposta crítica.   
O que irei fazer é:  
- Vou pegar os 25 melhores filmes com base nas pontuações de similaridade e calcular o voto do filme do 60º percentil.  
- Em seguida, usando isso como o valor de  $m$  , calcularemos a classificação ponderada de cada filme usando a fórmula do IMDB.

In [ ]:
# Define a função que usa a formula de classificação ponderada do IMDB
# Onde :
#       v é o número de votos para o filme
#       m é o mínimo de votos necessários para ser listado no gráfico
#       R é a classificação média do filme
#       C é a média de votos em todo o relatório  

# FALAR SOBRE ESSA SITUAÇÃO CM O PROFESSOR
# vote_counts = md_filmes[md_filmes['vote_count'].notnull()]['vote_count'].astype('int')
# vote_averages = md_filmes[md_filmes['vote_average'].notnull()]['vote_average'].astype('int')

# C = vote_averages.mean()
# m = vote_counts.quantile(0.95)

def classificacao_ponderada(x, C, m):
  v = x['vote_count']
  R = x['vote_average']

  return (v/(v+m) * R) + (m/(m+v) * C) 

In [ ]:
def obter_recomendacoes_melhoradas(title):
  # Pega o indice do filme com base no titulo em indices(mapa reverso)
  idx = indices[title]
  print(idx)
  # Cria uma lista com indice e similaridade de coseno baseado no indice do filme
  sim_scores = list(enumerate(cosine_sim[idx]))
  print(sim_scores)
  # Classifique os filmes com base nas pontuações de semelhança - decrescente
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  print(sim_scores)
  # Seleciona os 25 ids dos filmes e a similaridade de coseno
  sim_scores = sim_scores[1:26]
  print(sim_scores)
  # Cria uma lista contendo o id dos filmes - i[0]
  movie_indices = [i[0] for i in sim_scores]

  # Selecionando do df smd algumas colunas e os dados dos filmes que estão em movie indices
  movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
  # Filtra em movies['vote_count'] as linhas com valores que não são nulos e converte o tipo da coluna 
  vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
  # Filtra em movies['vote_average'] as linhas com valores que não são nulos e converte o tipo da coluna 
  vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
  # Retorna para C a média dos valores de vote_averages
  C = vote_averages.mean()
  # Retorna valores no quantil de vote_counts
  m = vote_counts.quantile(0.60)

  # Filtra os filmes que possuem quantidade de votos maior = a m e que vote_count e vote_average não são nulos
  mv_qualificado = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
  # Converte o tipo da coluna e atribui aos filmes qualificados
  mv_qualificado['vote_count'] = mv_qualificado['vote_count'].astype('int')
  # Converte o tipo da coluna e atribui aos filmes qualificados
  mv_qualificado['vote_average'] = mv_qualificado['vote_average'].astype('int')
  # Criar uma coluna com a classificação ponderada usando a formula do IMDB
  mv_qualificado['wr'] = mv_qualificado.apply(classificacao_ponderada, axis=1, args=(C, m))
  # Ordene os valores de forma decrescente com base na classificação ponderada 
  mv_qualificado = mv_qualificado.sort_values('wr', ascending=False).head(10)

  return mv_qualificado

In [ ]:
obter_recomendacoes_melhoradas('The Dark Knight')

6981
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.020965696734438367), (4, 0.0), (5, 0.08481041912882635), (6, 0.0), (7, 0.04774099160262887), (8, 0.04193139346887673), (9, 0.05017452060042545), (10, 0.020965696734438367), (11, 0.0), (12, 0.0), (13, 0.020965696734438367), (14, 0.020965696734438367), (15, 0.04318335170875306), (16, 0.01891511969643836), (17, 0.018490006540840973), (18, 0.02159167585437653), (19, 0.04193139346887673), (20, 0.047643061260321314), (21, 0.04078236951430929), (22, 0.06153846153846155), (23, 0.035438495596916704), (24, 0.016428801936338142), (25, 0.023870495801314433), (26, 0.020391184757154644), (27, 0.02303267198524394), (28, 0.0), (29, 0.039722906114947866), (30, 0.033449680400283635), (31, 0.018490006540840973), (32, 0.01891511969643836), (33, 0.020965696734438367), (34, 0.014720214798941532), (35, 0.023870495801314433), (36, 0.0), (37, 0.019861453057473933), (38, 0.024806946917841695), (39, 0.02159167585437653), (40, 0.08636670341750612), (41, 0.0209656967344383

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.872621
8613,Interstellar,11187,8,2014,7.843131
6623,The Prestige,4510,8,2006,7.662461
3381,Memento,4168,8,2000,7.641301
8031,The Dark Knight Rises,9263,7,2012,6.936548
6218,Batman Begins,7511,7,2005,6.923913
1134,Batman Returns,1706,6,1992,6.206438
132,Batman Forever,1529,5,1995,5.676520
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.224783
1260,Batman & Robin,1447,4,1997,5.167624


In [ ]:
obter_recomendacoes_melhoradas('Mean Girls')

5207
[(0, 0.024419314525275217), (1, 0.0), (2, 0.05751973085430508), (3, 0.027066598098038335), (4, 0.02564102564102564), (5, 0.0), (6, 0.027874733666903028), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.027066598098038335), (11, 0.027874733666903028), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.02387049580131443), (18, 0.027874733666903028), (19, 0.027066598098038335), (20, 0.02050230920261824), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.05264981264926564), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.021591675854376526), (31, 0.0), (32, 0.024419314525275217), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.02973505167250263), (37, 0.05128205128205128), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.02632490632463282), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.023357091793352585), (50, 0.0), (51, 0.02564102564102564), (52, 0.0), (53, 0.027066598098038335), (54, 0.027066598098038335), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.

,title,vote_count,vote_average,year,wr
1547,The Breakfast Club,2189,7,1985,6.753897
390,Dazed and Confused,588,7,1993,6.369346
8883,The DUFF,1372,6,2015,5.884127
3712,The Princess Diaries,1063,6,2001,5.860251
4763,Freaky Friday,919,6,2003,5.845406
6277,Just Like Heaven,595,6,2005,5.796851
6959,The Spiderwick Chronicles,593,6,2008,5.796457
7494,American Pie Presents: The Book of Love,454,5,2009,5.254920
7332,Ghosts of Girlfriends Past,716,5,2009,5.196974
7905,Mr. Popper's Penguins,775,5,2011,5.187382


In [ ]:
smd

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director,soup
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,7.7,5415.0,1995,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousi, toy, boy, friendship, friend, rival...",13,106,"[johnlasseter, johnlasseter, johnlasseter]",jealousi toy boy friendship friend rivalri boy...
1,1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,...,6.9,2413.0,1995,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgam, disappear, basedonchildren'sbook, n...",26,16,"[joejohnston, joejohnston, joejohnston]",boardgam disappear basedonchildren'sbook newho...
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,...,6.5,92.0,1995,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fish, bestfriend, duringcreditssting]",7,4,"[howarddeutch, howarddeutch, howarddeutch]",fish bestfriend duringcreditssting waltermatth...
3,3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,...,6.1,34.0,1995,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[basedonnovel, interracialrelationship, single...",10,10,"[forestwhitaker, forestwhitaker, forestwhitaker]",basedonnovel interracialrelationship singlemot...
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,...,5.7,173.0,1995,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[babi, midlifecrisi, confid, age, daughter, mo...",12,7,"[charlesshyer, charlesshyer, charlesshyer]",babi midlifecrisi confid age daughter motherda...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9214,40952,False,NaN,8000000,[Drama],NaN,159550,tt0255313,en,The Last Brickmaker in America,...,7.0,1.0,2001,"[sidneypoitier, wendycrewson, jayo.sanders]","[{'credit_id': '544475aac3a36819fb000578', 'de...",[friendship],7,2,"[greggchampion, greggchampion, greggchampion]",friendship sidneypoitier wendycrewson jayo.san...
9215,41172,False,NaN,1000000,"[Thriller, Romance]",NaN,392572,tt5165344,hi,रुस्तम,...,7.3,25.0,2016,"[akshaykumar, ileanad'cruz, eshagupta]","[{'credit_id': '5951baf692514129c4016600', 'de...",[bollywood],14,16,"[tinusureshdesai, tinusureshdesai, tinusureshd...",bollywood akshaykumar ileanad'cruz eshagupta t...
9216,41225,False,NaN,15050000,"[Adventure, Drama, History, Romance]",NaN,402672,tt3859980,hi,Mohenjo Daro,...,6.7,26.0,2016,"[hrithikroshan, poojahegde, kabirbedi]","[{'credit_id': '57cd5d3592514179d50018e8', 'de...",[bollywood],12,16,"[ashutoshgowariker, ashutoshgowariker, ashutos...",bollywood hrithikroshan poojahegde kabirbedi a...
9217,41391,False,NaN,15000000,"[Action, Adventure, Drama, Horror, Science Fic...",NaN,315011,tt4262980,ja,シン・ゴジラ,...,6.6,152.0,2016,"[hirokihasegawa, yutakatakenouchi, satomiishih...","[{'credit_id': '560892fa92514177550018b2', 'de...","[monster, godzilla, giantmonst, destruct, kaiju]",49,27,"[hideakianno, hideakianno, hideakianno]",monster godzilla giantmonst destruct kaiju hir...


#**FILTRAGEM COLABORATIVA**

In [ ]:
reader = Reader()

In [ ]:
avaliacoes = pd.read_csv('ratings_small.csv')
avaliacoes.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
dados = Dataset.load_from_df(avaliacoes[['userId', 'movieId', 'rating']], reader)
# kf = KFold(n_splits=5)
# kf.split(dados)

In [ ]:
svd = SVD()
cross_validate(svd, dados, measures=['RMSE', 'MAE'], verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8969  0.8891  0.8940  0.9030  0.9038  0.8973  0.0055  
MAE (testset)     0.6901  0.6841  0.6915  0.6968  0.6941  0.6913  0.0043  
Fit time          1.11    1.10    1.37    1.03    1.05    1.13    0.12    
Test time         0.13    0.11    0.13    0.13    0.12    0.12    0.00    


{'test_rmse': array([0.89687605, 0.88908901, 0.89400948, 0.90296144, 0.90379622]),
 'test_mae': array([0.69010137, 0.6841224 , 0.69146434, 0.69683483, 0.69411121]),
 'fit_time': (1.1051390171051025,
  1.0996599197387695,
  1.3730518817901611,
  1.0336658954620361,
  1.0497779846191406),
 'test_time': (0.1270742416381836,
  0.11474990844726562,
  0.12522077560424805,
  0.12727856636047363,
  0.12227964401245117)}

In [ ]:
trainset = dados.build_full_trainset()
svd.fit(trainset)

In [ ]:
avaliacoes[avaliacoes['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [ ]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.9262131035470826, details={'was_impossible': False})

# **FUNÇÕES UTILITARIAS**

In [ ]:
# Método que normaliza valores entre 0 e 1
# Recebe uma coluna como serie
def normalizacao_valores(serie):
  serie_index = serie.index
  #Fiz essa alteração

  array = serie.values
  y = array.reshape(-1, 1)
  scaler = MinMaxScaler(feature_range= (0, 1))
  rescaled = scaler.fit_transform(y)
  array = rescaled
  serie_normalizada = pd.DataFrame(array, index=serie_index)
  return serie_normalizada

In [ ]:
# Aqui as vezes retorna um int e as vezes retorna uma serie tratei dessa forma

def get_index(title):
  # Df indices contem o titulo e o código do filme ao lado
  idx = indices[title]

  if isinstance(idx, pd.Series):
    return idx[0]

  return idx

# **RECOMENDADOR KNN**  

Antes de qualquer coisa, ajustar o Dataset e organizar as colunas que irei utilizar o dataset de exemplo que irei me basear é esse:   
(https://github.com/krishnaik06/Recommendation_complete_tutorial/tree/master/KNN%20Movie%20Recommendation)  
Assim que estiver em Pedrinhas tenho que deixar o dataset que tenho conforme será utilizado aqui, realizar o pre-processado com os dados do meu Dataset.  
***Esse algoritmo utiliza a estratégia de filtragem colaborativa***  
Existem dois tipos:
* Baseado em itens;   
* Baseado em Usuários.  

***No nosso caso iremos utilizar a filtragem colaborativa baseado em usuários***

In [ ]:
def recomendacao_KNN(usuarioId):

  # Importando dataset
  ds_filmes = pd.read_csv('movies_metadata.csv', dtype={'imdb_id':"string"})
  ds_links = pd.read_csv('links.csv', dtype={'tmdbId':"Int64"})
  ds_avaliacoes = pd.read_csv('ratings_small.csv')

  # Remoção de linhas mal formatadas 
  ds_filmes = ds_filmes.drop([19730, 29503, 35587])

  # Verificando imdbids nulos
  ds_filmes[ds_filmes['imdb_id'].isnull()]

  # Preenchendo os imdb_ids faltantes
  for lista in ds_filmes.itertuples():
    if pd.isna(lista.imdb_id):
      ds_filmes.loc[lista.Index, 'imdb_id'] = 'tt' + str(ds_links.loc[ds_links['tmdbId'] == int(lista.id)]['imdbId'].values[0])

  # Extraindo coluna imdbId para pegar o código do filme em links
  ds_filmes['imdbId'] = ds_filmes['imdb_id'].apply(lambda x: x if pd.isna(x) else str(x)[2:])

  # Alterando tipo da coluna
  ds_filmes['imdbId'] = ds_filmes['imdbId'].astype('int')

  # Merge pegando os movieIds de Links
  ds_filmes = pd.merge(ds_filmes, ds_links, on='imdbId')

  # Removendo a coluna tmdbId
  ds_filmes.drop(['tmdbId'], axis=1, inplace=True)

  # Guardando dados da tabela full para utilizar na montagem do mapa reverso - REMOVER SE NÃO USAR
  # ds_filmes_fulltable = ds_filmes
  # ds_filmes_fulltable
  
  # Para fazer a filtragem colaborativa o que me interessa é o Id do Filme e o titulo
  # Selecionando/filtrandos as colunas que irei utilizar 
  ds_filmes = ds_filmes[['movieId', 'title']]
  

  # Filtrando as colunas que preciso
  ds_avaliacoes = ds_avaliacoes[['userId', 'movieId', 'rating']]
  

  # Juntar as duas bases, semelhante ao join no SQL 
  df_filmes_avaliacoes = pd.merge(ds_filmes, ds_avaliacoes, on='movieId')
  

  # Pega o id do usuário e coloca como indice da coluna e pivotar a tabela 
  # Onde os titulos apareçam em cima e as avaliações serão os valores 
  df_recommender = df_filmes_avaliacoes.pivot_table(index='userId', columns='title', values='rating').fillna(0)
  

  # Modelo dos vizinhos próximos - KNN
  # Distância do cosseno não sofre muito com a maldição da dimensionalidade
  # Instancia o modelo KNN, usando a metrica do cosseno(hiperparametros)
  modelo_knn = NearestNeighbors(metric='cosine')
  # Calcula todas as distâncias em relação aos vizinhos próximos
  modelo_knn.fit(df_recommender)

  # Quantidade de vizinhos mais próximos que quero
  qtde_vizinhos = 4

  # Recupera a linha que o usuário se encontra
  # Usuario que eu quero recomendar algo
  idx_usuario_knn = df_recommender.index.get_loc(df_recommender.loc[usuarioId].name)
  # Quando chamar o modelo de KNN preciso saber a distância dos vizinhos próximos
  # E quais são os indices vizinhos próximos
  # Irei passar no parâmetro de kneighbors() o registro que quero usar para puxar
  # Vizinho mais próximo - Fiz uma mudança ao invés de iloc usei loc
  distancia_vizinhos, indices_vizinhos = modelo_knn.kneighbors(df_recommender.iloc[idx_usuario_knn].values.reshape(1, -1), n_neighbors=qtde_vizinhos)


  usuario = df_recommender.index[idx_usuario_knn]

  # Cria a lista que irá conter os dataframes
  dfs_vizinhos_proximo = []

  # For para percorrer cada um da lista 
  # Usei o flatter para remover uma dimensão 
  for i in range(0, len(distancia_vizinhos.flatten())):
    # Pula o primeiro elemento pois vai ser ele mesmo
    if i == 0:
      #print("Usuário: {0}\n".format(usuario))
      ds_usuario_knn = df_recommender.loc[usuario].to_frame()
      dfs_vizinhos_proximo.append(ds_usuario_knn)
    else:
      #print("Vizinho {0} com distância {1}".format(df_recommender.index[indices_vizinhos.flatten()[i]], distancia_vizinhos.flatten()[i]))
      # Localiza o vizinho mais próximo em df_recommender
      vizinho_proximo = df_recommender.index[indices_vizinhos.flatten()[i]]
      # Adiciona o vizinho próximo a lista dos dataframes
      dfs_vizinhos_proximo.append(df_recommender.loc[vizinho_proximo].to_frame())
  
  # Lista que contem os dataframes dos vizinhos mais próximos 
  #dfs_vizinhos_proximo

  # Pegando o indice dos vizinhos mais próximos
  idx_vizinhos_prox = [col for i in range(len(dfs_vizinhos_proximo)) for col in dfs_vizinhos_proximo[i] if i != 0]
  
  # Faz o merge da dataframe do usuário com seus vizinhos
  ds_titulos = reduce(lambda df_esq, df_dir: pd.merge(df_esq, df_dir, on=['title']), dfs_vizinhos_proximo)
  # Ordena de forma descrescente pelos vizinhos mais próximos
  ds_titulos = ds_titulos.sort_values(by=idx_vizinhos_prox, ascending=False)
  
  # Filtrar todos os titulos do vizinho que foram > 0 e onde usuário não assistiu
  ds_titulos = ds_titulos[((ds_titulos[idx_vizinhos_prox[0]] > 0) | (ds_titulos[idx_vizinhos_prox[1]] > 0) | (ds_titulos[idx_vizinhos_prox[2]] > 0)) & (ds_titulos[usuario] == 0)]
  
  # Criando e calculando a média 
  ds_titulos['Media'] = (ds_titulos[idx_vizinhos_prox[0]] + ds_titulos[idx_vizinhos_prox[1]] + ds_titulos[idx_vizinhos_prox[2]]) / len(idx_vizinhos_prox)
  # Resetando o indice
  ds_titulos.reset_index(inplace=True)


  # Atribui a média os valores de média normalizados 
  ds_titulos['est'] = normalizacao_valores(ds_titulos['Media'])

  # Filtrando as colunas que irei devolver ao método híbrido 
  titulos_recomendacao_KNN = ds_titulos[['title', 'est']]
  titulos_recomendacao_KNN = titulos_recomendacao_KNN.sort_values(by=['est'], ascending=False)


  return titulos_recomendacao_KNN.head(10)


**Testando Recomendador KNN**

In [ ]:
recomendacao_KNN(1)

,title,est
0,Beverly Hills Cop,1.000000
13,The Godfather,0.642857
30,Titus,0.642857
1,The Brady Bunch Movie,0.642857
15,The Shawshank Redemption,0.642857
28,Proof,0.642857
14,The Godfather: Part II,0.642857
29,Three Kings,0.642857
31,A Perfect Murder,0.571429
32,Sylvia,0.571429


#**RECOMENDADOR HÍBRIDO**

In [ ]:
def convert_int(x):
  try:
    return int(x)
  except:
    return np.nan

In [ ]:
# Carregando o arquivo que contem os ids dos filmes, selecionando 
# as colunas 'movieId', 'tmdbId'
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]
# Converte a coluna id_map['tmdbId'] para inteiro
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
# Mudando os nomes da coluna tmdbId para id
id_map.columns = ['movieId', 'id']
# TENHO QUE VOLTAR PARA AQUI QUANDO CRIAR A VARIÁVEL SMD 
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [ ]:
indices_map = id_map.set_index('id')

In [ ]:
# Junta e soma as estimativas de cada dataframe 
def agrupa_recomendacoes(dataframe1, dataframe2):
  dfs_concatenados = pd.concat([dataframe1, dataframe2], ignore_index=True)
  dfs_agrupados = dfs_concatenados.groupby('title')[['est']].mean()
  # dfs_agrupados = dfs_concatenados.groupby('title')['est'].sum()

  dfs_agrupados = dfs_agrupados.sort_values('est', ascending=False)
  dfs_agrupados = dfs_agrupados.reset_index()

  return dfs_agrupados


In [ ]:
def recomendador_hibrido(userId, title):

  #Df indices contem o titulo e o código do filme ao lado
  # idx = indices[title]
  # print(idx)
  # tmdbId = id_map.loc[title]['id']
  # print(tmdbId)
  # movie_id = id_map.loc[title]['movieId']
  # print(movie_id)

  idx = get_index(title)

  titulos_recomendados_KNN = recomendacao_KNN(userId)
  

  sim_scores = list(enumerate(cosine_sim[int(idx)]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:26]
  movie_indices = [i[0] for i in sim_scores]

  movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
  movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
  movies['est'] = normalizacao_valores(movies['est'])
  movies = movies[['title', 'est']]

  movies = movies.sort_values('est', ascending=False)

  movies = movies.head(10)

  recomendacao_hibrida = agrupa_recomendacoes(titulos_recomendados_KNN, movies)

  return recomendacao_hibrida

In [ ]:
recomendador_hibrido(1, 'Avatar')

,title,est
0,Beverly Hills Cop,1.000000
1,The Terminator,1.000000
2,Aliens,0.894758
3,Star Trek Into Darkness,0.876324
4,Terminator 2: Judgment Day,0.876011
5,Fantastic Planet,0.826001
6,The Abyss,0.767210
7,Return from Witch Mountain,0.748404
8,Piranha Part Two: The Spawning,0.714579
9,X-Men: Days of Future Past,0.690067


In [ ]:
recomendador_hibrido(500, 'Avatar')

,title,est
0,Terminator 2: Judgment Day,1.000000
1,Walk the Line,1.000000
2,Apollo 13,0.920000
3,"I, Robot",0.920000
4,Piranha Part Two: The Spawning,0.906999
5,Spider-Man,0.880000
6,Rain Man,0.880000
7,Being John Malkovich,0.880000
8,Star Trek Into Darkness,0.871808
9,The Terminator,0.849765
